In [1]:
import SciServer.CasJobs as cj
import pandas
import numpy as np

In [2]:
pandas.set_option('display.max_rows', 999)
pandas.set_option('display.max_columns', 999)


In [3]:
sql="""
select * from openquery(MANGA,'
SELECT datname, pg_encoding_to_char(encoding) 
FROM pg_database') a
"""
df=cj.executeQuery(sql,"manga_postgres")
df.head()


,datname,pg_encoding_to_char
0,postgres,UTF8
1,swerner,UTF8
2,template1,UTF8
3,template0,UTF8
4,manga_local,UTF8


In [4]:
sql="""
select * from openquery(MANGA,'
select * from information_schema.columns 
where table_schema not in (''pg_catalog'', ''information_schema'',''public'')
') a
order by table_schema,table_name,ordinal_position
"""
df=cj.executeQuery(sql,"manga_postgres")
df.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,manga,mangaauxdb,cube_header,pk,1,nextval('mangaauxdb.cube_header_pk_seq'::regcl...,NO,integer,NaN,NaN,32.0,2.0,0.0,NaN,,None,,,,,,,,,,manga,pg_catalog,int4,,,,None,1,NO,NO,,,,,,NO,NEVER,,YES
1,manga,mangaauxdb,cube_header,header,2,,YES,json,NaN,NaN,NaN,NaN,NaN,NaN,,None,,,,,,,,,,manga,pg_catalog,json,,,,None,2,NO,NO,,,,,,NO,NEVER,,YES
2,manga,mangaauxdb,cube_header,cube_pk,3,,YES,integer,NaN,NaN,32.0,2.0,0.0,NaN,,None,,,,,,,,,,manga,pg_catalog,int4,,,,None,3,NO,NO,,,,,,NO,NEVER,,YES
3,manga,mangaauxdb,maskbit,pk,1,nextval('mangaauxdb.maskbit_pk_seq'::regclass),NO,integer,NaN,NaN,32.0,2.0,0.0,NaN,,None,,,,,,,,,,manga,pg_catalog,int4,,,,None,1,NO,NO,,,,,,NO,NEVER,,YES
4,manga,mangaauxdb,maskbit,flag,2,,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,,None,,,,,,,,,,manga,pg_catalog,text,,,,None,2,NO,NO,,,,,,NO,NEVER,,YES


In [5]:
def datatype(column, VENDOR='POSTGRES'):
    if VENDOR == 'POSTGRES':
        return postgres_datatype(column)
    elif VENDOR == "MYSQL":
        return mysqls_datatype(column)

def mysql_datatype(column):
    return column[1].upper()

def postgres_datatype(column):
    dt=column[1].upper()
    l=str(column[4]).upper()
    if (l != 'NAN') and not(np.isnan(float(l))):
        l=int(float(l))
    else:
        l=0
    if (dt in ['JSON','TEXT','ARRAY']) or (dt == 'CHARACTER VARYING' and l == 0):
        dt="NVARCHAR(MAX)"
    elif dt == 'CHARACTER VARYING':
        dt= f'NVARCHAR({l})'
    elif dt == 'BOOLEAN':
        dt="NVARCHAR(5)"
    elif dt == 'BYTEA':
        dt='VARBINARY(MAX)'
    elif dt.startswith('TIMESTAMP'):
        dt='DATETIME'
    elif dt == 'DOUBLE PRECISION':
        dt='FLOAT'
#     else if dt in ['INTEGER','SMALLINT','REAL','BIGINT']:
    return dt

In [6]:
current_op = 10000
tables=[]
schemas=[]
for i,row in df.iterrows():
    op = row['ordinal_position']
    if op < current_op:
        if row['table_schema'] not in schemas:
            schemas.append(row['table_schema'])
        table={'schema':row['table_schema'],'name':row['table_name'],'has_json':False,'has_array':False}
        tables.append(table)
        columns=[]
        table['columns']=columns
    current_op=op
    if row['data_type'].upper() == 'JSON':
        table['has_json'] = True
    if row['data_type'].upper() == 'ARRAY':
        table['has_array'] = True
    columns.append([row['column_name'],row['data_type'], row['is_nullable'],
                    row['character_maximum_length'],row['numeric_precision']
                    ,row['numeric_scale']])

In [8]:
with open('schemas.sql','w') as ft:
    for schema in schemas:
        ft.write('create schema '+schema+'\n\nGO\n' )

In [9]:
with open('postgres_views.sql','w') as vt:
    for table in tables:
        if not(table['has_array']):
            continue
        view='v_'+table['name']
            
#         i=f"""
# if exists (select 1 from information_schema.tables 
# where table_schema='{table['schema']}' and table_name='{view}')
# begin drop table {table['schema']}.{view} end\n\nGO\n\n"""
        
        ddl=f"""
CREATE VIEW {table['schema']}.{view} as
SELECT """
        ddlcolumns=[]
        for column in table['columns']:
            c=column[0]
            if column[1].upper().startswith("ARRAY"):
                c=f"array_to_json({c}) as {c}"
            ddlcolumns.append(f"{c}")
        ddl+="\n,      ".join(ddlcolumns)+"\n"
        ddl+=f"  FROM {table['schema']}.{table['name']};"+"\n"
        vt.write(ddl)
        print(ddl)


CREATE VIEW mangadapdb.v_modelspaxel as
SELECT pk
,      array_to_json(flux) as flux
,      array_to_json(ivar) as ivar
,      array_to_json(mask) as mask
,      array_to_json(model) as model
,      array_to_json(emline) as emline
,      array_to_json(emline_base) as emline_base
,      array_to_json(emline_mask) as emline_mask
,      x
,      y
,      modelcube_pk
,      binid_binned_spectra
,      binid_stellar_continua
,      binid_em_line_moments
,      binid_em_line_models
,      binid_spectral_indices
,      array_to_json(model_mask) as model_mask
,      array_to_json(stellar) as stellar
,      array_to_json(stellar_mask) as stellar_mask
  FROM mangadapdb.modelspaxel;


CREATE VIEW mangadapdb.v_redcorr as
SELECT pk
,      array_to_json(value) as value
,      modelcube_pk
  FROM mangadapdb.redcorr;


CREATE VIEW mangadatadb.v_cube as
SELECT pk
,      plate
,      mangaid
,      designid
,      pipeline_info_pk
,      wavelength_pk
,      ifudesign_pk
,      array_to_json(specres) 

In [10]:
with open('tables.sql','w') as ft, open('json_tables.sql','w') as fj, open('array_tables.sql','w') as fa:
    for table in tables:
        if table['has_array']:
            etable='v_'+table['name']
        else:
            etable=table['name']
            
        i=f"""
if exists (select 1 from information_schema.tables 
where table_schema='{table['schema']}' and table_name='{table['name']}')
begin drop external table {table['schema']}.[{table['name']}] end\n\nGO\n\n"""
        
        ddl=i+f"CREATE EXTERNAL TABLE {table['schema']}.[{table['name']}] (\n"
        ddlcolumns=[]
        for column in table['columns']:
            _null="NULL" if (column[2] == 'YES') else "NOT NULL"
            ddlcolumns.append(f"  [{column[0]}] {datatype(column)} {_null} -- {column[1]}")
        ddl+=" "+"\n,".join(ddlcolumns)+"\n)"
        ddl+=f"\nwith (location='manga.{table['schema']}.[{etable}]',data_source=pg64)\n\nGO\n\n"
        if table['has_array']:
            fa.write(ddl)
        elif table['has_json']:
            fj.write(ddl)
        else:
            ft.write(ddl)
        print(ddl)


if exists (select 1 from information_schema.tables 
where table_schema='mangaauxdb' and table_name='cube_header')
begin drop external table mangaauxdb.[cube_header] end

GO

CREATE EXTERNAL TABLE mangaauxdb.[cube_header] (
   [pk] INTEGER NOT NULL -- integer
,  [header] NVARCHAR(MAX) NULL -- json
,  [cube_pk] INTEGER NULL -- integer
)
with (location='manga.mangaauxdb.[cube_header]',data_source=pg64)

GO



if exists (select 1 from information_schema.tables 
where table_schema='mangaauxdb' and table_name='maskbit')
begin drop external table mangaauxdb.[maskbit] end

GO

CREATE EXTERNAL TABLE mangaauxdb.[maskbit] (
   [pk] INTEGER NOT NULL -- integer
,  [flag] NVARCHAR(MAX) NULL -- text
,  [bit] INTEGER NULL -- integer
,  [label] NVARCHAR(MAX) NULL -- text
,  [description] NVARCHAR(MAX) NULL -- text
)
with (location='manga.mangaauxdb.[maskbit]',data_source=pg64)

GO



if exists (select 1 from information_schema.tables 
where table_schema='mangaauxdb' and table_name='maskbit_labels')
be